# 1. COMPLETE SCRIPT (Extraction → Cleaning → Enrichment → CSV)

In [20]:
import requests
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta, timezone

# -----------------------------
# Date range: last 5 years
# -----------------------------
end_date = datetime.now(timezone.utc)
start_date = end_date - timedelta(days=5 * 365)

# -----------------------------
# USGS API endpoint
# -----------------------------
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"

params = {
    "format": "geojson",
    "starttime": start_date.strftime("%Y-%m-%d"),
    "endtime": end_date.strftime("%Y-%m-%d"),
    "minmagnitude": 3,
    "limit": 20000
}

response = requests.get(url, params=params)
response.raise_for_status()
data = response.json()

# -----------------------------
# Extract records
# -----------------------------
records = []

for feature in data["features"]:
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]

    records.append({
        "id": feature["id"],

        "time": pd.to_datetime(props.get("time"), unit="ms", errors="coerce"),
        "updated": pd.to_datetime(props.get("updated"), unit="ms", errors="coerce"),

        "latitude": coords[1],
        "longitude": coords[0],
        "depth_km": coords[2],

        "mag": props.get("mag"),
        "magType": props.get("magType"),
        "place": props.get("place"),
        "status": props.get("status"),
        "tsunami": props.get("tsunami"),
        "sig": props.get("sig"),

        "net": props.get("net"),
        "nst": props.get("nst"),
        "dmin": props.get("dmin"),
        "rms": props.get("rms"),
        "gap": props.get("gap"),

        "magError": props.get("magError"),
        "depthError": props.get("depthError"),
        "magNst": props.get("magNst"),

        "locationSource": props.get("locationSource"),
        "magSource": props.get("magSource"),

        "types": props.get("types"),
        "ids": props.get("ids"),
        "sources": props.get("sources"),
        "type": props.get("type")
    })

# -----------------------------
# Create DataFrame
# -----------------------------
df = pd.DataFrame(records)

print("Rows :", df.shape[0])
print("Columns :", df.shape[1])
print(df.head())


Rows : 20000
Columns : 26
             id                    time                 updated  latitude  \
0    us7000rs0p 2026-01-25 22:02:10.618 2026-01-26 01:55:14.040   23.3940   
1    us7000rs0i 2026-01-25 20:29:01.866 2026-01-25 20:46:45.040   -8.6248   
2  ak2026btoyfc 2026-01-25 19:51:55.287 2026-01-25 20:23:58.827   67.6520   
3    us7000rs08 2026-01-25 19:24:40.117 2026-01-25 19:40:39.040  -31.4355   
4    us7000rs01 2026-01-25 17:56:53.285 2026-01-25 18:18:13.040   37.5901   

   longitude  depth_km  mag magType                          place    status  \
0   143.4290    10.000  5.0      mb  Volcano Islands, Japan region  reviewed   
1   -74.6666   122.560  4.6      mb     16 km NNE of Honoria, Peru  reviewed   
2  -166.6220     5.500  3.8      ml  77 km S of Point Hope, Alaska  reviewed   
3   -71.5445    41.926  4.5     mwr    41 km WNW of Illapel, Chile  reviewed   
4    21.4946    36.351  4.1      mb     4 km S of Epitálio, Greece  reviewed   

   ...    gap  magError depthE

# 2. Clean Text Fields

In [22]:
# -----------------------------
# Extract country from place
# -----------------------------
def extract_country(place):
    if pd.isna(place):
        return None
    parts = place.split(",")
    return parts[-1].strip() if len(parts) > 1 else None

df["country"] = df["place"].apply(extract_country)

# -----------------------------
# Normalize alert field (if exists)
# -----------------------------
if "alert" in df.columns:
    df["alert"] = df["alert"].astype(str).str.lower().replace("nan", None)

# -----------------------------
# Clean string fields
# -----------------------------
text_cols = [
    "magType", "status", "type", "net",
    "sources", "types", "locationSource", "magSource"
]

for col in text_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.lower()
        .replace("nan", None)
    )


# 3. Clean Numeric Fields

In [23]:
numeric_cols = [
    "mag", "depth_km", "nst", "dmin", "rms",
    "gap", "magError", "depthError", "magNst", "sig"
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Fill missing numeric values
df[numeric_cols] = df[numeric_cols].fillna(0)


# 4. Add derived columns (Date Components, Depth Classification, Magnitude Impact Classification)

In [24]:
df["year"] = df["time"].dt.year
df["month"] = df["time"].dt.month
df["day"] = df["time"].dt.day
df["day_of_week"] = df["time"].dt.day_name()


In [25]:
df["depth_category"] = np.where(
    df["depth_km"] <= 70, "shallow",
    np.where(df["depth_km"] <= 300, "intermediate", "deep")
)


In [26]:
def magnitude_flag(mag):
    if mag >= 7.0:
        return "destructive"
    elif mag >= 5.0:
        return "strong"
    elif mag >= 3.0:
        return "moderate"
    else:
        return "weak"

df["impact_level"] = df["mag"].apply(magnitude_flag)


In [27]:
# df.to_csv(
#     "earthquakes_clean_last_5_years.csv",
#     index=False,
#     encoding="utf-8-sig"
# )

# print("CSV exported successfully")

CSV exported successfully


In [7]:
# df.to_csv(
#    "earthquakes_workbench.csv",
#    index=False,
#    encoding="utf-8-sig"  # IMPORTANT
# )

In [29]:
def to_ascii(val):
    if isinstance(val, str):
        return val.encode("ascii", errors="ignore").decode("ascii")
    return val

df_ascii = df.map(to_ascii)

df_ascii.to_csv(
    "earthquakes_clean_last_5_years1.csv",
    index=False,
    encoding="ascii"
)

In [ ]:
from google.colab import files
files.download("earthquakes_clean_last_5_years1.csv")